In [1]:
from qrisp import QuantumVariable, h, x, prepare, conjugate
from qrisp.vqe.problems.heisenberg import create_heisenberg_init_function, heisenberg_problem, create_heisenberg_hamiltonian
import networkx as nx
import numpy as np
import scipy
import matplotlib.pyplot as plt
from run_QITE import run_QITE

import pickle
from time import time
from datetime import datetime

def save_data(data, filename):  
    # Open a file for writing
    with open(filename+'.pickle', 'wb') as file:
        # Write the object to the file
        pickle.dump(data, file)

def load_data(filename):
    with open(filename+'.pickle', 'rb') as file:
        # Load the object from the file
        data = pickle.load(file)
    return data

In [2]:
L = 10
G = nx.Graph()
G.add_edges_from([(k,(k+1)%L) for k in range(L-1)]) 
# nx.draw(G, with_labels=True)

In [ ]:
from qrisp.operators import X, Y, Z

J = 1
B = 0.5
H = create_heisenberg_hamiltonian(G, J, B)
print(H)

# Define scaling factor
F = 10

# Hamiltonian simulation via first order Suzuki-Trotter formula with 5 steps
def exp_H(qv, t):
    H.trotterization(method='commuting')(qv,t/F,5)

# Hamiltonian simulation via second order Suzuki-Trotter formula with 2 steps
def exp_H_2(qv, t):
    H.trotterization(order=2,method='commuting')(qv,t/F,2)

X(0)*X(1) + X(1)*X(2) + X(2)*X(3) + X(3)*X(4) + X(4)*X(5) + X(5)*X(6) + X(6)*X(7) + X(7)*X(8) + X(8)*X(9) + Y(0)*Y(1) + Y(1)*Y(2) + Y(2)*Y(3) + Y(3)*Y(4) + Y(4)*Y(5) + Y(5)*Y(6) + Y(6)*Y(7) + Y(7)*Y(8) + Y(8)*Y(9) + 0.5*Z(0) + Z(0)*Z(1) + 0.5*Z(1) + Z(1)*Z(2) + 0.5*Z(2) + Z(2)*Z(3) + 0.5*Z(3) + Z(3)*Z(4) + 0.5*Z(4) + Z(4)*Z(5) + 0.5*Z(5) + Z(5)*Z(6) + 0.5*Z(6) + Z(6)*Z(7) + 0.5*Z(7) + Z(7)*Z(8) + 0.5*Z(8) + Z(8)*Z(9) + 0.5*Z(9)


## QITE with singlet, plus, vqe warm start

In [5]:
# Tensor product of singlet states
M = nx.maximal_matching(G)
U_singlet = create_heisenberg_init_function(M)

# VQE
vqe = heisenberg_problem(G,J,B, ansatz_type='per hamiltonian')
U_vqe = vqe.train_function(QuantumVariable(G.number_of_nodes()),depth=1,max_iter=20)

results = []

results.append(run_QITE(H, U_singlet, exp_H_2, np.linspace(.01,1.5,20), 5, method='GC', use_statevectors=True))
results.append(run_QITE(H, U_singlet, exp_H_2, np.linspace(.01,1.5,20), 4, method='HOPF', use_statevectors=True))

results.append(run_QITE(H, U_vqe, exp_H_2, np.linspace(.01,1.5,20), 5, method='GC', use_statevectors=True))
results.append(run_QITE(H, U_vqe, exp_H_2, np.linspace(.01,1.5,20), 4, method='HOPF', use_statevectors=True))

In [7]:
# Save benchmark data
current_datetime = datetime.now()
date_time_string = current_datetime.strftime("%m-%d-%H")
save_data(results,'data/benchmarks_2nd_B=0.5_'+date_time_string)